# Global Entry Interivew Scheduler

Scrapes the [TTP website](https://ttp.cbp.dhs.gov/schedulerui/schedule-interview/location?lang=en&vo=true&returnUrl=ttp-external&service=up) to determine if the desired location has any availability. Then sends a push notification to mobile device indicating whether there is availability or not. Uses Pushover because it has a one month free trial and only $5 for a permanent license (as of March 2025).

## Setup

0. On your machine, install `python<=3.10`, `selenium`, `webdriver-manager`, and `pushover`. Unfortunately, Pushover only works in `python<=3.10` due to dependence on `SafeConfigParser`
1. On your mobile device, download the Pushover app on mobile device and create a Pushover account
2. On the Pushover website, login and create a new application to generate an API token
3. On your machine, modify `scheduler_config.ini` file with your user id and API token

## Usage

1.  After setup, modify `scheduler_config.ini` with the desired location number. Although the location number is not visible on the TTP website, you can find the location number by inspecting the web element of the location (e.g. Boise Enrollment Center is 120).

## Example Config File

The config file `scheduler_config.ini` should look something like
```
[pushover]
user_key = <your Pushover user id>
app_key = <your API token>
[location]
location=120
```

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
from datetime import datetime
import pushover
import configparser

In [8]:
def send_pushover_notification(configfile, message, title="Global Entry Appointment", sound="pushover"):
    """Sends a push notification to your iPhone using Pushover."""
    try:
        client = pushover.PushoverClient(configfile=configfile)
        client.send_message(message, title=title, sound=sound)
        print("Pushover notification sent successfully!")

    except Exception as e:
        print(f"An error occurred: {e}")

In [9]:
configfile='scheduler_config.ini'
webpage = "https://ttp.cbp.dhs.gov/schedulerui/schedule-interview/location?lang=en&vo=true&returnUrl=ttp-external&service=up"
accessed = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")

config = configparser.ConfigParser()
config.read(configfile)
location = int(config.get('location','location'))

In [10]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get(webpage)
wait = WebDriverWait(driver, 5)
chicago_link = wait.until(EC.element_to_be_clickable((By.XPATH, f"//*[@id='centerDetailsUS{location}']/strong/span")))
chicago_link.click()
time.sleep(2)
popup = wait.until(EC.presence_of_element_located((By.XPATH, f"//*[@id='popoverUS{location}']/div/div/div/strong")))
popup_text = popup.find_element(By.XPATH, f"ancestor::div[@id='popoverUS{location}']").text
location_name = popup_text.split('\n')[0]
availability = " ".join(popup_text.replace('\n',' ').split('Google Map')[-1].strip().split(' ')[:6])
status = "No Appointments" if "full" in availability else "Appointments Available!"

In [13]:
output = f"""{status}\n{availability}\nAccessed: {accessed}\nLocation: {location_name}\n{webpage}"""

In [14]:
print(output)

Appointments Available!
Next Available Appointment: August 13, 2025
Accessed: 2025-03-09T16:28:45
Location: Boise Enrollment Center
https://ttp.cbp.dhs.gov/schedulerui/schedule-interview/location?lang=en&vo=true&returnUrl=ttp-external&service=up


In [8]:
send_pushover_notification(message=output,configfile=configfile)

Pushover notification sent successfully!
